In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ccxt
from pathlib import Path




Bad key "text.kerning_factor" on line 4 in
C:\Users\drbri\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def in_return_cov(df): 
    price = df.copy()
    price = (price / price.shift(1)) -1
    price.iloc[0,:] = 0 # Primeira linha = 0     
    
    returns = np.matrix(price)
    mean_returns = np.mean(returns, axis = 0)
    
    annual_returns = np.array([])
    for i in range(len(np.transpose(mean_returns))):
        annual_returns = np.append(annual_returns,(mean_returns[0,i]+1)**252-1)   
    
    cov = price.cov()*252
    
    return (annual_returns, np.matrix(cov))

#Carrega tabela de preços
def load_prices(symbol,
                index,
                start_date,
                end_date):
    
    dates = pd.date_range(start_date, end_date)
    dfBase = pd.DataFrame(index = dates)
    path = "C:/Users/Guilherme Overney/Desktop/DATA/{}.csv"
    
    dfIndex = pd.read_csv(path.format(index),index_col = "Date",
                          parse_dates = True,usecols = ['Date', 'Adj Close'], na_values=['nan'])
    
    dfIndex = dfIndex.dropna()
    dfIndex = dfIndex.rename(columns={'Adj Close': '{}'.format(index)})
    dfBase = dfBase.join(dfIndex, how = 'inner')
    
    
    for symbol in symbols:
        df_temp = pd.read_csv(path.format(symbol), index_col = "Date", parse_dates = True,
                              usecols = ['Date', 'Adj Close'], na_values=['nan'])
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        dfBase = dfBase.join(df_temp)
    
    dfBase.fillna(method = "ffill", inplace = True)
    dfBase.fillna(method = "bfill", inplace = True)
    
    return dfBase.drop('IBOV',axis = 1) #Tiro o índice da tabela de preços...

#Calcula o retorno de um portfólio de ativos
def port_return(W,r):
    return sum(W*r)

#Calcula o peso inicial do portfolio dado o tamanho de mercado do papel
def mkt_weights(weights):
    return np.array(weights) / sum(weights)

def risk_return(W,S,r,rf=0.06):
    var = np.dot(np.dot(W,S),np.transpose(W))
    port_r = port_return(W,r)
    return np.ndarray.item((port_r - rf) / var)

#Calcula a matriz de retorno implicito
def vector_equilibrium_return(S, W, r ):
    
    A = risk_return(W,S,r)
    
    return (np.dot(A,np.dot(S,W)))

def diago_omega(t, P, S):
    
    omega = np.dot(t,np.dot(P,np.dot(S,np.transpose(P))))
    
    for i in range(len(omega)):
        for y in range(len(omega)):
            if i != y: omega[i,y] = 0
    return omega
    
#Black Litterman Model
def posterior_estimate_return(t,S,P,Q,PI):
    
    omega = diago_omega(t, P, S)
    
    parte_1 = t*np.dot(S,np.transpose(P))
    parte_2 = np.linalg.inv(np.dot(P*t,np.dot(S,np.transpose(P))) + omega)
    parte_3 = Q - np.dot(P,np.transpose(PI))
    
    return np.transpose(PI) + np.dot(parte_1,np.dot(parte_2,parte_3))

def posterior_covariance(t,S,P,PI):
    
    omega = diago_omega(t, P, S)
    
    parte_1 = t*np.dot(S,np.transpose(P))
    parte_2 = np.linalg.inv(t*np.dot(P,np.dot(S,np.transpose(P)))+omega)
    parte_3 = t*np.dot(P,S)

    return t*S - np.dot(parte_1,np.dot(parte_2,parte_3))

In [3]:
data=Path('Classification.csv')
cdf=pd.read_csv(data)

In [4]:
symbols=[]
ave_risk=1
if ave_risk==1:
    symbols=cdf['symbol'].loc[(cdf['class']==2)|(cdf['class']==0)].to_list()
elif ave_risk==2:
    symbols=cdf['symbol'].to_list()
elif ave_risk==3:
    symbols=cdf['symbol'].loc[(cdf['class']==1)|(cdf['class']==0)].to_list()


In [5]:
def get_prices(symbols):
    i=0
    exchange = ccxt.kraken({
    'apiKey': KRAKEN_API_KEY,
    'secret': KRAKEN_SECRET_API_KEY,
    })
    since = exchange.parse8601('2018-01-01T00:00:00z')

    for symbol in symbols:
        if i==0:
            try:
                data = exchange.fetchOHLCV(symbol,'1d', since, params = {})
                header = ['Date', 'Open', 'High', 'Low', symbol, 'Volume']
                ticker_df = pd.DataFrame(data, columns=header)
                ticker_df['Date']=pd.to_datetime(ticker_df.Date/1000, unit='s')
                ticker_df.set_index('Date', inplace=True)
                ticker_df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
                i+=1
            except:
                continue
        else:
            try:
                data = exchange.fetchOHLCV(symbol,'1d', since, params = {})
                header = ['Date', 'Open', 'High', 'Low', symbol, 'Volume']
                df = pd.DataFrame(data, columns=header)
                df['Date']=pd.to_datetime(df.Date/1000, unit='s')
                df.set_index('Date', inplace=True)
                df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
                ticker_df=pd.concat([ticker_df, df], axis=1, join='inner')
            except:
                continue
    return(ticker_df)

In [6]:
ticker_df=get_prices(symbols)

In [7]:
tickers=ticker_df.columns.tolist()
data_df=pd.DataFrame()
data_df['tickers']=tickers
data_df['mrkcap']=0
data_df['returns']=0
data_df['ann_ret']=0
data_df['price']=0
data_df.set_index('tickers',inplace=True)


In [8]:
assets_return , S= in_return_cov(ticker_df)
data_df['returns']=assets_return

In [9]:
cdf.set_index('symbol', inplace=True)


In [10]:
pret=ticker_df.pct_change()
SD=pret.std()* np.sqrt(252)
ave_ret=pret.mean()* 252
last_price=ticker_df.iloc[-1,:]


In [11]:
for ticker in tickers:
    data_df.loc[ticker,'mrkcap']=cdf.loc[ticker, 'Market_Cap']
    data_df.loc[ticker,'ann_ret']=ave_ret.loc[ticker]
    data_df.loc[ticker,'price']=last_price.loc[ticker]
    data_df.loc[ticker,'SD']=SD.loc[ticker]
data_df.dropna(inplace=True)
clean_ticker=data_df.index.to_list()


In [12]:
for ticker in tickers:
    if ticker not in clean_ticker:
        ticker_df.drop([ticker], axis=1, inplace=True)


In [13]:
assets_return , S= in_return_cov(ticker_df)


In [14]:
W = mkt_weights(data_df['mrkcap'])

In [15]:
A = risk_return(W,S,data_df['returns'])


In [16]:
PI = vector_equilibrium_return(S, W, data_df['returns'])


In [17]:
N=len(clean_ticker)
Q = np.zeros((N,1))
P = np.zeros((N,N))
for n in range(len(clean_ticker)):
    P[n,n]=1
    Q[n,0]=data_df.iloc[n,3]

In [18]:
t = 1 / len(ticker_df) # Medida de incerteza das estimativas de retornos pelo método Meucci
expc_return = posterior_estimate_return(t,S,P,Q,PI)


In [19]:
post_cov = posterior_covariance(t,S,P,PI)


In [20]:

cov_post_estimate = post_cov + S


In [21]:
new_weigth = np.dot(np.transpose(expc_return),np.linalg.inv(A*cov_post_estimate))
weights_df=pd.DataFrame(new_weigth, columns = clean_ticker).T


In [22]:
port_df=weights_df.loc[weights_df[0]>0]

In [23]:
port_df=port_df/port_df[0].sum()

In [24]:
port_df.sum()
port_df.columns=['weights']

In [25]:
port_df['price']=0
port_df['return']=0
port_df['SD']=0
port_df['shares']=0
port_df['value']=0
port_tickers=port_df.index.to_list()

In [26]:
for ticker in clean_ticker:
    if ticker not in port_tickers:
        ticker_df.drop([ticker], axis=1, inplace=True)

In [27]:
amount=1000000

for ticker in port_tickers:
    port_df.loc[ticker,'price']=data_df.loc[ticker,'price']
    port_df.loc[ticker,'SD']=data_df.loc[ticker,'SD']
    port_df.loc[ticker,'return']=data_df.loc[ticker,'ann_ret']
    port_df.loc[ticker,'shares']=(amount*port_df.loc[ticker, 'weights']//port_df.loc[ticker, 'price'])
    port_df.loc[ticker,'value']=(port_df.loc[ticker, 'shares']*port_df.loc[ticker, 'price'])  
left=amount-port_df['value'].sum()

In [28]:
wts=port_df['weights'].to_list()
wts=np.asarray(wts)


In [29]:
port_ret=ticker_df.pct_change()
port_ret.dropna(inplace=True)
port_ret['return']=port_ret.mul(wts,axis=1).sum(axis=1)


In [30]:
# =pret.std()* np.sqrt(252)
ann_ret=port_ret['return'].mean()* 252-.0062

In [31]:
cov_matrix = port_ret.iloc[:,0:-1].cov()


In [32]:
ann_port_var=round(((np.dot(wts.T,np.dot(cov_matrix, wts)))* np.sqrt(252)),3)
ann_port_std=np.sqrt(ann_port_var)

In [33]:
sharpe=ann_ret/ann_port_std

In [35]:
metrics=[ann_ret, sharpe, ann_port_var]
metrics

[-0.5420599675041692, -2.500336183228597, 0.047]